In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import malaya_speech
import json
import tensorflow as tf
import malaya_boilerplate

malaya_boilerplate.__version__

'0.0.24rc1'

In [3]:
model = malaya_speech.stt.deep_transducer(model = 'conformer-stack-2mixed', device = 'gpu:0')

Downloading:   0%|          | 0.00/125M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

2023-01-27 15:34:46.017561: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 15:34:46.023533: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 15:34:46.027590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 15:34:46.028355: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [4]:
import malaya_speech
import json

with open('/home/husein/ssd1/speech-bahasa/malay-asr-test.json') as fopen:
    test_set = json.load(fopen)

In [5]:
y, sr = malaya_speech.load(test_set['X'][0])
y1, sr = malaya_speech.load(test_set['X'][1])

In [6]:
model.greedy_decoder([y, y1])

2023-01-27 15:34:58.894854: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-01-27 15:34:59.284557: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2023-01-27 15:34:59.766172: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


['ms ialah penyakit yang mempengaruhi sistem saraf pusat yang dibentuk daripada otak saraf tunjang dan saraf optik',
 'msla penyakit yang mempengaruhi sistem saraf yang dibentuk daripada kota saraf tunjang dan saraf optik']

In [7]:
batch_size = 3

In [8]:
import kenlm
from pyctcdecode.language_model import LanguageModel

In [9]:
kenlm_model = kenlm.Model('model.klm')
language_model = LanguageModel(kenlm_model, alpha = 0.01, beta = 0.5)

In [10]:
model.beam_decoder_lm([y, y1], language_model)

['ms ialah penyakit yang mempengaruhi sistem saraf pusat yang dibentuk daripada otak saraf tunjang dan zarah optik',
 'ms ialah penyakit yang mempengaruhi sistem saraf yang dibentuk daripada kota saraf tunjang dan saraf optik']

In [11]:
malaya_speech.metrics.calculate_wer, malaya_speech.metrics.calculate_cer

(<function malaya_speech.utils.metrics.calculate_wer(actual, hyp)>,
 <function malaya_speech.utils.metrics.calculate_cer(actual, hyp)>)

In [12]:
from tqdm import tqdm

wer, cer = [], []
wer_lm, cer_lm = [], []

for i in tqdm(range(0, len(test_set['X']), batch_size)):
    batch = test_set['X'][i: i + batch_size]
    ys = [malaya_speech.load(k)[0] for k in batch]
    greedy_decoded = model.greedy_decoder(ys)
    lm_decoded = model.beam_decoder_lm(ys, language_model)
    
    texts = test_set['Y'][i: i + batch_size]
    
    for d, t in zip(greedy_decoded, texts):
        wer.append(malaya_speech.metrics.calculate_wer(t, d))
        cer.append(malaya_speech.metrics.calculate_cer(t, d))
        
    for d, t in zip(lm_decoded, texts):
        wer_lm.append(malaya_speech.metrics.calculate_wer(t, d))
        cer_lm.append(malaya_speech.metrics.calculate_cer(t, d))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 247/247 [41:43<00:00, 10.14s/it]


In [15]:
import numpy as np

In [16]:
np.mean(wer), np.mean(cer), np.mean(wer_lm), np.mean(cer_lm)

(0.24483694813923196,
 0.11740932775605861,
 0.2224221577664085,
 0.10805900747363882)